In [1]:
import pandas as pd

def RenomearColunasPercentuais(df):
    novo_nome = {}

    for col in df.columns:

        if col.startswith("Quantidade Produção - Minério ROM (t) - "):
            mineral = col.replace("Quantidade Produção - Minério ROM (t) - ", "")
            novo_nome[col] = f"% Produção - {mineral}"

        
        elif col.startswith("Quantidade Venda (t) - "):
            mineral = col.replace("Quantidade Venda (t) - ", "")
            novo_nome[col] = f"% Venda - {mineral}"

        
        elif col.startswith("Valor Venda (R$) - "):
            mineral = col.replace("Valor Venda (R$) - ", "")
            novo_nome[col] = f"% Valor Venda - {mineral}"
        
        else:
            continue


    df = df.rename(columns=novo_nome)
    return df

def PegarValoresSubstancia(df, colunas):
    Df_Pivotado = df.pivot_table(
        index=['UF', 'Ano base'],
        columns='Substância Mineral',
        values=colunas,
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    # Ajusta os nomes das colunas vindas do pivot (multi-index)
    Df_Pivotado.columns = [
        f"{c[0]} - {c[1]}" if isinstance(c, tuple) else c
        for c in Df_Pivotado.columns
    ]
    
    return Df_Pivotado


def converterValores(df):
    for col in df.columns:
         if col.startswith("% Produção"):
            df[col] = (df[col]/df['Quantidade Produção - Minério ROM (t)']*100).round(2)
         elif col.startswith("% Venda"):
            df[col] = (df[col]/df['Quantidade Venda (t)']*100).round(2)
         elif col.startswith("% Valor Venda"):
            df[col] = (df[col]/df['Valor Venda (R$)']*100).round(2)
         else:
             continue
    return df
    

In [2]:
Producao_Bruta = pd.read_csv('../Prata/Producao_Bruta.csv',encoding='latin')
Df_Final = Producao_Bruta[['UF','Quantidade Produção - Minério ROM (t)','Ano base','Quantidade Venda (t)','Valor Venda (R$)']].groupby(['UF','Ano base'],as_index=False).sum()
Df_tmp = Producao_Bruta[['UF','Quantidade Produção - Minério ROM (t)','Ano base','Quantidade Venda (t)','Valor Venda (R$)','Substância Mineral']].groupby(['UF','Ano base','Substância Mineral'],as_index=False).sum()

colunas = ['Quantidade Produção - Minério ROM (t)','Quantidade Venda (t)','Valor Venda (R$)']
Df_tmp = PegarValoresSubstancia(Df_tmp,colunas)
Df_tmp = RenomearColunasPercentuais(Df_tmp)
Df_tmp.columns = Df_tmp.columns.str.strip().str.replace("-", "").str.strip()

Df_Final = pd.merge(left=Df_Final,right=Df_tmp,left_on=['UF','Ano base'],right_on=['UF','Ano base'],how='left')
Df_Final = converterValores(Df_Final)

for valor in colunas:
    Df_Final[valor] = Df_Final[valor].round(2)
cols_finais = []

cols_finais += ['Ano base', 'UF', 'Quantidade Produção - Minério ROM (t)']

cols_finais += [c for c in Df_Final.columns if "% Produção" in c]

cols_finais += ['Quantidade Venda (t)']

cols_finais += [c for c in Df_Final.columns if "% Venda" in c]

cols_finais += ['Valor Venda (R$)']

cols_finais += [c for c in Df_Final.columns if "% Valor Venda" in c]

Df_Final = Df_Final[cols_finais]

Df_Final.to_csv("../Ouro/Producao_Bruta.csv",index=False)
